In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
from keras.models import Sequential
from keras.layers import LSTM, Dense, SimpleRNN, Input, Activation, Dropout
from keras import backend as K
from tensorflow.keras.optimizers import Adam,SGD
import tensorflow as tf
from keras.models import Model
from keras.models import load_model
import time #to calculate the computation time

2022-10-13 01:28:24.383110: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-13 01:28:24.383147: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Collect data for the target process

In [2]:
# specifying constant parameters
#input varioable: Q, C_A0
#state variable: T, C_A

T_0 = 300
V = 1
k_0 = 8.46*(np.power(10,6))
C_p = 0.231
rho_L = 1000
F = 5
E = 5*(np.power(10,4))
delta_H = -1.15*(np.power(10,4))
R = 8.314

C_A0s = 4   # the steady state for input variable C_A0
Q_s = 0.0  # the steady state for input variable Q

C_As = 1.95 # the steady state for state variable C_A
T_s = 402  # the steady state for state variable T

t_final = 0.01  #the control period
t_step = 1e-4   # the step to use first-principle to calculate the state
P = np.array([[1060, 22], [22, 0.52]])

In [3]:
# generating inputs and initial states for CSTR, all expressed in deviation form

# Test 1 full stability region
u1_list = np.linspace(-3.5, 3.5, 10, endpoint=True)
u2_list = np.linspace(-5e5, 5e5, 10, endpoint=True)
T_initial = np.linspace(300, 550, 15, endpoint=True) - T_s
CA_initial = np.linspace(1, 4., 15, endpoint=True) - C_As    # CA and T must be >0
#print(T_initial)
#control variable: C_A0, Q
#state variable: C_A, T

# sieve out initial states that lie outside of stability region

T_start = list()
CA_start = list()

for T in T_initial:   
    for CA in CA_initial:
        x = np.array([CA, T])
        if x @ P @ x < 368:   #calculate the stability region for state 
            CA_start.append(CA)
            T_start.append(T)
print("number of initial conditions: {}".format(len(CA_start)))

# convert to np.arrays
CA_start = np.array([CA_start])
T_start = np.array([T_start])
print(CA_start.shape)
x_deviation = np.concatenate((CA_start.T, T_start.T), axis=1)  # every row is a pair of initial states within stability region
print("shape of x_deviation is {}".format(x_deviation.shape))
print(x_deviation.shape)  # the initial state is in 

number of initial conditions: 32
(1, 32)
shape of x_deviation is (32, 2)
(32, 2)


In [4]:
def CSTR_simulation(F, V, C_A0, k_0, E, R, T_0, delta_H, rho_L, C_p, Q, t_final, t_step, C_A_initial, T_initial):
    """
        simulating CSTR using forward Euler method
    """
    
    C_A_list = list()  # evolution of CA over time
    T_list = list()  # evolution of T over time
    
    C_A = C_A_initial + C_As  # the real state.the derivation plus the steady state
    T = T_initial + T_s
    
    for i in range(int(t_final / t_step)):
        dCAdt = F / V * (C_A0 - C_A) - k_0 * np.exp(-E / (R * T)) * C_A**2
        dTdt = F / V * (T_0 - T) - delta_H / (rho_L * C_p) * k_0 * np.exp(-E / (R * T)) * C_A**2 + Q / (rho_L * C_p * V)
        
        C_A += dCAdt * t_step
        T += dTdt * t_step
        
        if i%5 ==0:
            C_A_list.append(C_A - C_As)  # in deviation form
            T_list.append(T - T_s)  # in deviation form 
    
    return C_A_list, T_list

In [5]:
# get X and y data for training and testing

CA_output = list()
T_output = list()

CA_input = list()
T_input = list()
CA0_input = list()
Q_input = list()   #input variable for 

for u1 in u1_list:
    C_A0 = u1 + C_A0s
    
    for u2 in u2_list:
        Q = u2 + Q_s
        
        for C_A_initial, T_initial in x_deviation:
            CA0_input.append(u1)
            Q_input.append(u2)
            CA_input.append(C_A_initial)
            T_input.append(T_initial)
            
            C_A_list, T_list = CSTR_simulation(F, V, C_A0, k_0, E, R, T_0, delta_H, rho_L, C_p, Q, t_final, t_step, C_A_initial, T_initial)
            CA_output.append(C_A_list)
            T_output.append(T_list)

In [6]:
# collate input for RNN

CA0_input = np.array(CA0_input)
CA0_input = CA0_input.reshape(-1,1,1)

Q_input = np.array(Q_input)
Q_input = Q_input.reshape(-1,1,1)

CA_input = np.array(CA_input)
CA_input = CA_input.reshape(-1,1,1)

T_input = np.array(T_input)
T_input = T_input.reshape(-1,1,1)

RNN_input = np.concatenate((T_input, CA_input, Q_input, CA0_input), axis=2)   #the value for input variable and the initial value for state variable 

"""
    the input to RNN is in the shape [number of samples x timestep x variables], and the input variables are same for every
    time step, not sure if my treatment here is correct
"""
print("RNN_input shape is {}".format(RNN_input.shape))
RNN_input = RNN_input.repeat(20, axis=1)  # to keep consensus with the shape for RNN_output, since the output variable is collected 100(0.01/1e-4) times for each RNN_input
print("RNN_input shape is {}".format(RNN_input.shape))

RNN_input shape is (3200, 1, 4)
RNN_input shape is (3200, 20, 4)


In [7]:
# collate output for RNN

CA_output = np.array(CA_output)
CA_output = CA_output.reshape(-1, 20, 1)

T_output = np.array(T_output)
T_output = T_output.reshape(-1, 20, 1)

RNN_output = np.concatenate((T_output, CA_output), axis=2)
print("RNN_output shape is {}".format(RNN_output.shape))  # output shape: number of samples x timestep x variables

RNN_output shape is (3200, 20, 2)


In [8]:
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(RNN_input, RNN_output, test_size=0.4, random_state=123)

# define scalers for both X and y base on training data only
scaler_X = preprocessing.StandardScaler().fit(X_train.reshape(-1, 4))
scaler_y = preprocessing.StandardScaler().fit(y_train.reshape(-1, 2))


X_train = scaler_X.transform(X_train.reshape(-1, 4)).reshape(-1,20,4)
X_test = scaler_X.transform(X_test.reshape(-1, 4)).reshape(-1,20,4)
y_train = scaler_y.transform(y_train.reshape(-1,2)).reshape(-1,20,2)
y_test = scaler_y.transform(y_test.reshape(-1,2)).reshape(-1,20,2)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1920, 20, 4) (1280, 20, 4) (1920, 20, 2) (1280, 20, 2)


# Develop standard RNN model for the target process

In [9]:
model2 = Sequential()
model2.add(SimpleRNN(32, activation='tanh', return_sequences=True))
model2.add(SimpleRNN(32, activation='tanh', return_sequences=True))
model2.add(Dense(2, activation='linear'))
model2.compile(optimizer='adam', loss='mse', metrics=['mse'])

t0 = time.time()
history = model2.fit(X_train, y_train, epochs=600, batch_size=256, validation_split=0.25, verbose=2)
t1 = time.time()
print(t1-t0)

Epoch 1/600


2022-10-13 01:30:19.288998: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-10-13 01:30:19.289039: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-13 01:30:19.289065: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c895b2a47570): /proc/driver/nvidia/version does not exist
2022-10-13 01:30:19.289228: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


6/6 - 1s - loss: 2.2836 - mse: 2.2836 - val_loss: 1.0710 - val_mse: 1.0710 - 1s/epoch - 180ms/step
Epoch 2/600
6/6 - 0s - loss: 0.6744 - mse: 0.6744 - val_loss: 0.3503 - val_mse: 0.3503 - 81ms/epoch - 14ms/step
Epoch 3/600
6/6 - 0s - loss: 0.3007 - mse: 0.3007 - val_loss: 0.2644 - val_mse: 0.2644 - 75ms/epoch - 13ms/step
Epoch 4/600
6/6 - 0s - loss: 0.2481 - mse: 0.2481 - val_loss: 0.2230 - val_mse: 0.2230 - 75ms/epoch - 13ms/step
Epoch 5/600
6/6 - 0s - loss: 0.1969 - mse: 0.1969 - val_loss: 0.1724 - val_mse: 0.1724 - 76ms/epoch - 13ms/step
Epoch 6/600
6/6 - 0s - loss: 0.1525 - mse: 0.1525 - val_loss: 0.1438 - val_mse: 0.1438 - 80ms/epoch - 13ms/step
Epoch 7/600
6/6 - 0s - loss: 0.1311 - mse: 0.1311 - val_loss: 0.1273 - val_mse: 0.1273 - 85ms/epoch - 14ms/step
Epoch 8/600
6/6 - 0s - loss: 0.1145 - mse: 0.1145 - val_loss: 0.1087 - val_mse: 0.1087 - 80ms/epoch - 13ms/step
Epoch 9/600
6/6 - 0s - loss: 0.0996 - mse: 0.0996 - val_loss: 0.0962 - val_mse: 0.0962 - 75ms/epoch - 12ms/step
Epoch

Epoch 74/600
6/6 - 0s - loss: 0.0081 - mse: 0.0081 - val_loss: 0.0083 - val_mse: 0.0083 - 79ms/epoch - 13ms/step
Epoch 75/600
6/6 - 0s - loss: 0.0080 - mse: 0.0080 - val_loss: 0.0082 - val_mse: 0.0082 - 76ms/epoch - 13ms/step
Epoch 76/600
6/6 - 0s - loss: 0.0079 - mse: 0.0079 - val_loss: 0.0081 - val_mse: 0.0081 - 75ms/epoch - 12ms/step
Epoch 77/600
6/6 - 0s - loss: 0.0078 - mse: 0.0078 - val_loss: 0.0080 - val_mse: 0.0080 - 71ms/epoch - 12ms/step
Epoch 78/600
6/6 - 0s - loss: 0.0077 - mse: 0.0077 - val_loss: 0.0079 - val_mse: 0.0079 - 71ms/epoch - 12ms/step
Epoch 79/600
6/6 - 0s - loss: 0.0076 - mse: 0.0076 - val_loss: 0.0078 - val_mse: 0.0078 - 75ms/epoch - 13ms/step
Epoch 80/600
6/6 - 0s - loss: 0.0075 - mse: 0.0075 - val_loss: 0.0077 - val_mse: 0.0077 - 87ms/epoch - 14ms/step
Epoch 81/600
6/6 - 0s - loss: 0.0074 - mse: 0.0074 - val_loss: 0.0076 - val_mse: 0.0076 - 84ms/epoch - 14ms/step
Epoch 82/600
6/6 - 0s - loss: 0.0072 - mse: 0.0072 - val_loss: 0.0075 - val_mse: 0.0075 - 86ms/e

6/6 - 0s - loss: 0.0035 - mse: 0.0035 - val_loss: 0.0036 - val_mse: 0.0036 - 70ms/epoch - 12ms/step
Epoch 147/600
6/6 - 0s - loss: 0.0034 - mse: 0.0034 - val_loss: 0.0035 - val_mse: 0.0035 - 71ms/epoch - 12ms/step
Epoch 148/600
6/6 - 0s - loss: 0.0034 - mse: 0.0034 - val_loss: 0.0035 - val_mse: 0.0035 - 82ms/epoch - 14ms/step
Epoch 149/600
6/6 - 0s - loss: 0.0034 - mse: 0.0034 - val_loss: 0.0035 - val_mse: 0.0035 - 84ms/epoch - 14ms/step
Epoch 150/600
6/6 - 0s - loss: 0.0033 - mse: 0.0033 - val_loss: 0.0034 - val_mse: 0.0034 - 71ms/epoch - 12ms/step
Epoch 151/600
6/6 - 0s - loss: 0.0033 - mse: 0.0033 - val_loss: 0.0034 - val_mse: 0.0034 - 70ms/epoch - 12ms/step
Epoch 152/600
6/6 - 0s - loss: 0.0033 - mse: 0.0033 - val_loss: 0.0034 - val_mse: 0.0034 - 70ms/epoch - 12ms/step
Epoch 153/600
6/6 - 0s - loss: 0.0033 - mse: 0.0033 - val_loss: 0.0033 - val_mse: 0.0033 - 71ms/epoch - 12ms/step
Epoch 154/600
6/6 - 0s - loss: 0.0032 - mse: 0.0032 - val_loss: 0.0033 - val_mse: 0.0033 - 71ms/epoch 

...

Epoch 290/600
6/6 - 0s - loss: 0.0012 - mse: 0.0012 - val_loss: 0.0013 - val_mse: 0.0013 - 71ms/epoch - 12ms/step
Epoch 291/600
6/6 - 0s - loss: 0.0012 - mse: 0.0012 - val_loss: 0.0013 - val_mse: 0.0013 - 76ms/epoch - 13ms/step
Epoch 292/600
6/6 - 0s - loss: 0.0012 - mse: 0.0012 - val_loss: 0.0012 - val_mse: 0.0012 - 92ms/epoch - 15ms/step
Epoch 293/600
6/6 - 0s - loss: 0.0012 - mse: 0.0012 - val_loss: 0.0012 - val_mse: 0.0012 - 97ms/epoch - 16ms/step
Epoch 294/600
6/6 - 0s - loss: 0.0012 - mse: 0.0012 - val_loss: 0.0012 - val_mse: 0.0012 - 90ms/epoch - 15ms/step
Epoch 295/600
6/6 - 0s - loss: 0.0012 - mse: 0.0012 - val_loss: 0.0012 - val_mse: 0.0012 - 78ms/epoch - 13ms/step
Epoch 296/600
6/6 - 0s - loss: 0.0012 - mse: 0.0012 - val_loss: 0.0012 - val_mse: 0.0012 - 93ms/epoch - 15ms/step
Epoch 297/600
6/6 - 0s - loss: 0.0012 - mse: 0.0012 - val_loss: 0.0012 - val_mse: 0.0012 - 82ms/epoch - 14ms/step
Epoch 298/600
6/6 - 0s - loss: 0.0012 - mse: 0.0012 - val_loss: 0.0012 - val_mse: 0.0012

Epoch 359/600
6/6 - 0s - loss: 9.0270e-04 - mse: 9.0270e-04 - val_loss: 9.2808e-04 - val_mse: 9.2808e-04 - 73ms/epoch - 12ms/step
Epoch 360/600
6/6 - 0s - loss: 8.8476e-04 - mse: 8.8476e-04 - val_loss: 9.0869e-04 - val_mse: 9.0869e-04 - 77ms/epoch - 13ms/step
Epoch 361/600
6/6 - 0s - loss: 8.6785e-04 - mse: 8.6785e-04 - val_loss: 8.9601e-04 - val_mse: 8.9601e-04 - 86ms/epoch - 14ms/step
Epoch 362/600
6/6 - 0s - loss: 8.7022e-04 - mse: 8.7022e-04 - val_loss: 8.8508e-04 - val_mse: 8.8508e-04 - 76ms/epoch - 13ms/step
Epoch 363/600
6/6 - 0s - loss: 8.5988e-04 - mse: 8.5988e-04 - val_loss: 8.8928e-04 - val_mse: 8.8928e-04 - 75ms/epoch - 12ms/step
Epoch 364/600
6/6 - 0s - loss: 8.8207e-04 - mse: 8.8207e-04 - val_loss: 8.7388e-04 - val_mse: 8.7388e-04 - 76ms/epoch - 13ms/step
Epoch 365/600
6/6 - 0s - loss: 8.6925e-04 - mse: 8.6925e-04 - val_loss: 8.7049e-04 - val_mse: 8.7049e-04 - 71ms/epoch - 12ms/step
Epoch 366/600
6/6 - 0s - loss: 8.7358e-04 - mse: 8.7358e-04 - val_loss: 8.7464e-04 - val_m

6/6 - 0s - loss: 6.7960e-04 - mse: 6.7960e-04 - val_loss: 6.9612e-04 - val_mse: 6.9612e-04 - 71ms/epoch - 12ms/step
Epoch 423/600
6/6 - 0s - loss: 6.7579e-04 - mse: 6.7579e-04 - val_loss: 6.9674e-04 - val_mse: 6.9674e-04 - 77ms/epoch - 13ms/step
Epoch 424/600
6/6 - 0s - loss: 6.7606e-04 - mse: 6.7606e-04 - val_loss: 6.8707e-04 - val_mse: 6.8707e-04 - 77ms/epoch - 13ms/step
Epoch 425/600
6/6 - 0s - loss: 6.6596e-04 - mse: 6.6596e-04 - val_loss: 6.9402e-04 - val_mse: 6.9402e-04 - 76ms/epoch - 13ms/step
Epoch 426/600
6/6 - 0s - loss: 6.6693e-04 - mse: 6.6693e-04 - val_loss: 7.2784e-04 - val_mse: 7.2784e-04 - 79ms/epoch - 13ms/step
Epoch 427/600
6/6 - 0s - loss: 6.8667e-04 - mse: 6.8667e-04 - val_loss: 6.8133e-04 - val_mse: 6.8133e-04 - 79ms/epoch - 13ms/step
Epoch 428/600
6/6 - 0s - loss: 6.7721e-04 - mse: 6.7721e-04 - val_loss: 6.6946e-04 - val_mse: 6.6946e-04 - 69ms/epoch - 12ms/step
Epoch 429/600
6/6 - 0s - loss: 6.9106e-04 - mse: 6.9106e-04 - val_loss: 6.9862e-04 - val_mse: 6.9862e-04

Epoch 486/600
6/6 - 0s - loss: 5.8533e-04 - mse: 5.8533e-04 - val_loss: 5.9195e-04 - val_mse: 5.9195e-04 - 70ms/epoch - 12ms/step
Epoch 487/600
6/6 - 0s - loss: 5.8204e-04 - mse: 5.8204e-04 - val_loss: 5.5113e-04 - val_mse: 5.5113e-04 - 71ms/epoch - 12ms/step
Epoch 488/600
6/6 - 0s - loss: 5.4994e-04 - mse: 5.4994e-04 - val_loss: 5.7071e-04 - val_mse: 5.7071e-04 - 77ms/epoch - 13ms/step
Epoch 489/600
6/6 - 0s - loss: 5.5970e-04 - mse: 5.5970e-04 - val_loss: 5.4355e-04 - val_mse: 5.4355e-04 - 74ms/epoch - 12ms/step
Epoch 490/600
6/6 - 0s - loss: 5.5227e-04 - mse: 5.5227e-04 - val_loss: 5.8500e-04 - val_mse: 5.8500e-04 - 78ms/epoch - 13ms/step
Epoch 491/600
6/6 - 0s - loss: 5.5826e-04 - mse: 5.5826e-04 - val_loss: 5.4946e-04 - val_mse: 5.4946e-04 - 71ms/epoch - 12ms/step
Epoch 492/600
6/6 - 0s - loss: 5.4938e-04 - mse: 5.4938e-04 - val_loss: 5.5352e-04 - val_mse: 5.5352e-04 - 78ms/epoch - 13ms/step
Epoch 493/600
6/6 - 0s - loss: 5.4764e-04 - mse: 5.4764e-04 - val_loss: 5.4014e-04 - val_m

6/6 - 0s - loss: 4.8632e-04 - mse: 4.8632e-04 - val_loss: 5.5161e-04 - val_mse: 5.5161e-04 - 69ms/epoch - 12ms/step
Epoch 550/600
6/6 - 0s - loss: 4.9220e-04 - mse: 4.9220e-04 - val_loss: 5.3957e-04 - val_mse: 5.3957e-04 - 71ms/epoch - 12ms/step
Epoch 551/600
6/6 - 0s - loss: 5.0694e-04 - mse: 5.0694e-04 - val_loss: 4.5918e-04 - val_mse: 4.5918e-04 - 73ms/epoch - 12ms/step
Epoch 552/600
6/6 - 0s - loss: 4.8192e-04 - mse: 4.8192e-04 - val_loss: 4.7451e-04 - val_mse: 4.7451e-04 - 71ms/epoch - 12ms/step
Epoch 553/600
6/6 - 0s - loss: 4.6668e-04 - mse: 4.6668e-04 - val_loss: 5.2934e-04 - val_mse: 5.2934e-04 - 81ms/epoch - 13ms/step
Epoch 554/600
6/6 - 0s - loss: 4.8808e-04 - mse: 4.8808e-04 - val_loss: 4.5621e-04 - val_mse: 4.5621e-04 - 84ms/epoch - 14ms/step
Epoch 555/600
6/6 - 0s - loss: 4.6650e-04 - mse: 4.6650e-04 - val_loss: 4.5510e-04 - val_mse: 4.5510e-04 - 78ms/epoch - 13ms/step
Epoch 556/600
6/6 - 0s - loss: 4.5116e-04 - mse: 4.5116e-04 - val_loss: 4.5204e-04 - val_mse: 4.5204e-04

In [10]:
#use the test data to evaluate the model
loss_and_metrics = model2.evaluate(X_test, y_test, batch_size=256)
print(loss_and_metrics)

print(t1-t0)
model2.save('model_n.h5')

5/5 [==============================] - 0s 6ms/step - loss: 4.0898e-04 - mse: 4.0898e-04
[0.0004089761059731245, 0.0004089761059731245]
48.70114183425903


# Use adapter tuning method to develop target model from pre-trained model

In [11]:
#load the basic model trained before, one hidden layer with 32 ne
basic = load_model('basic.h5')

In [12]:
#use the test data to evaluate the model the generalization ability
loss_and_metrics = basic.evaluate(X_test, y_test, batch_size=256)
print(loss_and_metrics)


5/5 [==============================] - 0s 5ms/step - loss: 0.0071 - mse: 0.0071
[0.00705911498516798, 0.00705911498516798]


In [13]:
#copy the basic model
basic_u1 = tf.keras.models.clone_model(basic)
basic_u1.set_weights(basic.get_weights())

inputs = tf.keras.Input(shape=(20,4))
basic_u1.layers[0].trainable = False   #Freeze the layer from pre-trained model
x = basic_u1.layers[0](inputs)
x1 = tf.keras.layers.SimpleRNN(32, activation='tanh', return_sequences=True)(x)
x2 = tf.keras.layers.Dense(2, activation='linear')(x1)

transf_model1 = tf.keras.Model(inputs=inputs,outputs=x2)

In [14]:
print(transf_model1.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20, 4)]           0         
                                                                 
 simple_rnn (SimpleRNN)      (None, 20, 32)            1184      
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 20, 32)            2080      
                                                                 
 dense_1 (Dense)             (None, 20, 2)             66        
                                                                 
Total params: 3,330
Trainable params: 2,146
Non-trainable params: 1,184
_________________________________________________________________
None


In [15]:
transf_model1.compile(optimizer='adam', loss='mse', metrics=['mse'])
t0 = time.time()
history1 = transf_model1.fit(X_train, y_train, epochs=300, batch_size=256, validation_split=0.25, verbose=2)
t1 = time.time()

print(t1-t0)

Epoch 1/300
6/6 - 1s - loss: 0.7413 - mse: 0.7413 - val_loss: 0.4678 - val_mse: 0.4678 - 870ms/epoch - 145ms/step
Epoch 2/300
6/6 - 0s - loss: 0.3468 - mse: 0.3468 - val_loss: 0.2278 - val_mse: 0.2278 - 70ms/epoch - 12ms/step
Epoch 3/300
6/6 - 0s - loss: 0.1869 - mse: 0.1869 - val_loss: 0.1553 - val_mse: 0.1553 - 71ms/epoch - 12ms/step
Epoch 4/300
6/6 - 0s - loss: 0.1476 - mse: 0.1476 - val_loss: 0.1326 - val_mse: 0.1326 - 63ms/epoch - 10ms/step
Epoch 5/300
6/6 - 0s - loss: 0.1187 - mse: 0.1187 - val_loss: 0.0979 - val_mse: 0.0979 - 66ms/epoch - 11ms/step
Epoch 6/300
6/6 - 0s - loss: 0.0881 - mse: 0.0881 - val_loss: 0.0776 - val_mse: 0.0776 - 65ms/epoch - 11ms/step
Epoch 7/300
6/6 - 0s - loss: 0.0731 - mse: 0.0731 - val_loss: 0.0673 - val_mse: 0.0673 - 62ms/epoch - 10ms/step
Epoch 8/300
6/6 - 0s - loss: 0.0626 - mse: 0.0626 - val_loss: 0.0562 - val_mse: 0.0562 - 64ms/epoch - 11ms/step
Epoch 9/300
6/6 - 0s - loss: 0.0526 - mse: 0.0526 - val_loss: 0.0483 - val_mse: 0.0483 - 66ms/epoch - 

Epoch 74/300
6/6 - 0s - loss: 0.0027 - mse: 0.0027 - val_loss: 0.0027 - val_mse: 0.0027 - 71ms/epoch - 12ms/step
Epoch 75/300
6/6 - 0s - loss: 0.0026 - mse: 0.0026 - val_loss: 0.0027 - val_mse: 0.0027 - 73ms/epoch - 12ms/step
Epoch 76/300
6/6 - 0s - loss: 0.0026 - mse: 0.0026 - val_loss: 0.0026 - val_mse: 0.0026 - 70ms/epoch - 12ms/step
Epoch 77/300
6/6 - 0s - loss: 0.0026 - mse: 0.0026 - val_loss: 0.0026 - val_mse: 0.0026 - 62ms/epoch - 10ms/step
Epoch 78/300
6/6 - 0s - loss: 0.0025 - mse: 0.0025 - val_loss: 0.0026 - val_mse: 0.0026 - 61ms/epoch - 10ms/step
Epoch 79/300
6/6 - 0s - loss: 0.0025 - mse: 0.0025 - val_loss: 0.0025 - val_mse: 0.0025 - 63ms/epoch - 11ms/step
Epoch 80/300
6/6 - 0s - loss: 0.0024 - mse: 0.0024 - val_loss: 0.0025 - val_mse: 0.0025 - 67ms/epoch - 11ms/step
Epoch 81/300
6/6 - 0s - loss: 0.0024 - mse: 0.0024 - val_loss: 0.0024 - val_mse: 0.0024 - 69ms/epoch - 11ms/step
Epoch 82/300
6/6 - 0s - loss: 0.0024 - mse: 0.0024 - val_loss: 0.0024 - val_mse: 0.0024 - 80ms/e

6/6 - 0s - loss: 0.0012 - mse: 0.0012 - val_loss: 0.0013 - val_mse: 0.0013 - 63ms/epoch - 11ms/step
Epoch 147/300
6/6 - 0s - loss: 0.0012 - mse: 0.0012 - val_loss: 0.0013 - val_mse: 0.0013 - 68ms/epoch - 11ms/step
Epoch 148/300
6/6 - 0s - loss: 0.0012 - mse: 0.0012 - val_loss: 0.0013 - val_mse: 0.0013 - 85ms/epoch - 14ms/step
Epoch 149/300
6/6 - 0s - loss: 0.0012 - mse: 0.0012 - val_loss: 0.0012 - val_mse: 0.0012 - 81ms/epoch - 14ms/step
Epoch 150/300
6/6 - 0s - loss: 0.0012 - mse: 0.0012 - val_loss: 0.0012 - val_mse: 0.0012 - 84ms/epoch - 14ms/step
Epoch 151/300
6/6 - 0s - loss: 0.0012 - mse: 0.0012 - val_loss: 0.0012 - val_mse: 0.0012 - 88ms/epoch - 15ms/step
Epoch 152/300
6/6 - 0s - loss: 0.0012 - mse: 0.0012 - val_loss: 0.0012 - val_mse: 0.0012 - 77ms/epoch - 13ms/step
Epoch 153/300
6/6 - 0s - loss: 0.0012 - mse: 0.0012 - val_loss: 0.0012 - val_mse: 0.0012 - 81ms/epoch - 14ms/step
Epoch 154/300
6/6 - 0s - loss: 0.0012 - mse: 0.0012 - val_loss: 0.0012 - val_mse: 0.0012 - 97ms/epoch 

Epoch 214/300
6/6 - 0s - loss: 7.9684e-04 - mse: 7.9684e-04 - val_loss: 8.1454e-04 - val_mse: 8.1454e-04 - 66ms/epoch - 11ms/step
Epoch 215/300
6/6 - 0s - loss: 7.9308e-04 - mse: 7.9308e-04 - val_loss: 8.1052e-04 - val_mse: 8.1052e-04 - 64ms/epoch - 11ms/step
Epoch 216/300
6/6 - 0s - loss: 7.8863e-04 - mse: 7.8863e-04 - val_loss: 8.0938e-04 - val_mse: 8.0938e-04 - 64ms/epoch - 11ms/step
Epoch 217/300
6/6 - 0s - loss: 7.8390e-04 - mse: 7.8390e-04 - val_loss: 8.0056e-04 - val_mse: 8.0056e-04 - 71ms/epoch - 12ms/step
Epoch 218/300
6/6 - 0s - loss: 7.8071e-04 - mse: 7.8071e-04 - val_loss: 7.9979e-04 - val_mse: 7.9979e-04 - 74ms/epoch - 12ms/step
Epoch 219/300
6/6 - 0s - loss: 7.7517e-04 - mse: 7.7517e-04 - val_loss: 7.9414e-04 - val_mse: 7.9414e-04 - 74ms/epoch - 12ms/step
Epoch 220/300
6/6 - 0s - loss: 7.7077e-04 - mse: 7.7077e-04 - val_loss: 7.8751e-04 - val_mse: 7.8751e-04 - 68ms/epoch - 11ms/step
Epoch 221/300
6/6 - 0s - loss: 7.6795e-04 - mse: 7.6795e-04 - val_loss: 7.9217e-04 - val_m

6/6 - 0s - loss: 5.8657e-04 - mse: 5.8657e-04 - val_loss: 6.0161e-04 - val_mse: 6.0161e-04 - 69ms/epoch - 12ms/step
Epoch 278/300
6/6 - 0s - loss: 5.8428e-04 - mse: 5.8428e-04 - val_loss: 5.9776e-04 - val_mse: 5.9776e-04 - 68ms/epoch - 11ms/step
Epoch 279/300
6/6 - 0s - loss: 5.8094e-04 - mse: 5.8094e-04 - val_loss: 5.9443e-04 - val_mse: 5.9443e-04 - 67ms/epoch - 11ms/step
Epoch 280/300
6/6 - 0s - loss: 5.7833e-04 - mse: 5.7833e-04 - val_loss: 5.9314e-04 - val_mse: 5.9314e-04 - 66ms/epoch - 11ms/step
Epoch 281/300
6/6 - 0s - loss: 5.7659e-04 - mse: 5.7659e-04 - val_loss: 5.9122e-04 - val_mse: 5.9122e-04 - 66ms/epoch - 11ms/step
Epoch 282/300
6/6 - 0s - loss: 5.7497e-04 - mse: 5.7497e-04 - val_loss: 5.9080e-04 - val_mse: 5.9080e-04 - 68ms/epoch - 11ms/step
Epoch 283/300
6/6 - 0s - loss: 5.7250e-04 - mse: 5.7250e-04 - val_loss: 5.8367e-04 - val_mse: 5.8367e-04 - 69ms/epoch - 12ms/step
Epoch 284/300
6/6 - 0s - loss: 5.6893e-04 - mse: 5.6893e-04 - val_loss: 5.8135e-04 - val_mse: 5.8135e-04

In [16]:
#use the test data to evaluate the model
loss_and_metrics = transf_model1.evaluate(X_test, y_test, batch_size=256)
print(loss_and_metrics)

print(t1-t0)

5/5 [==============================] - 0s 6ms/step - loss: 5.5080e-04 - mse: 5.5080e-04
[0.0005508001195266843, 0.0005508001195266843]
22.586060285568237


In [17]:
#unfreeze the basic model and do the fine-tuning for the whole Model
transf_model1.layers[1].trainable = True
print(transf_model1.summary())


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20, 4)]           0         
                                                                 
 simple_rnn (SimpleRNN)      (None, 20, 32)            1184      
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 20, 32)            2080      
                                                                 
 dense_1 (Dense)             (None, 20, 2)             66        
                                                                 
Total params: 3,330
Trainable params: 3,330
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
t0 = time.time()
history1 = transf_model1.fit(X_train, y_train, epochs=300, batch_size=256, validation_split=0.25, verbose=2)
t1 = time.time()

print(t1 - t0) #memory the computation time

loss_and_metrics = transf_model1.evaluate(X_test, y_test, batch_size=128)
print(loss_and_metrics)

Epoch 1/300
6/6 - 0s - loss: 5.3500e-04 - mse: 5.3500e-04 - val_loss: 5.4365e-04 - val_mse: 5.4365e-04 - 85ms/epoch - 14ms/step
Epoch 2/300
6/6 - 0s - loss: 5.3039e-04 - mse: 5.3039e-04 - val_loss: 5.3935e-04 - val_mse: 5.3935e-04 - 75ms/epoch - 12ms/step
Epoch 3/300
6/6 - 0s - loss: 5.2759e-04 - mse: 5.2759e-04 - val_loss: 5.3982e-04 - val_mse: 5.3982e-04 - 70ms/epoch - 12ms/step
Epoch 4/300
6/6 - 0s - loss: 5.2838e-04 - mse: 5.2838e-04 - val_loss: 5.4279e-04 - val_mse: 5.4279e-04 - 81ms/epoch - 14ms/step
Epoch 5/300
6/6 - 0s - loss: 5.2466e-04 - mse: 5.2466e-04 - val_loss: 5.3715e-04 - val_mse: 5.3715e-04 - 87ms/epoch - 14ms/step
Epoch 6/300
6/6 - 0s - loss: 5.2048e-04 - mse: 5.2048e-04 - val_loss: 5.3170e-04 - val_mse: 5.3170e-04 - 83ms/epoch - 14ms/step
Epoch 7/300
6/6 - 0s - loss: 5.1887e-04 - mse: 5.1887e-04 - val_loss: 5.2945e-04 - val_mse: 5.2945e-04 - 95ms/epoch - 16ms/step
Epoch 8/300
6/6 - 0s - loss: 5.1782e-04 - mse: 5.1782e-04 - val_loss: 5.2981e-04 - val_mse: 5.2981e-04 -

Epoch 65/300
6/6 - 0s - loss: 4.1870e-04 - mse: 4.1870e-04 - val_loss: 4.3003e-04 - val_mse: 4.3003e-04 - 64ms/epoch - 11ms/step
Epoch 66/300
6/6 - 0s - loss: 4.1727e-04 - mse: 4.1727e-04 - val_loss: 4.2738e-04 - val_mse: 4.2738e-04 - 63ms/epoch - 10ms/step
Epoch 67/300
6/6 - 0s - loss: 4.1734e-04 - mse: 4.1734e-04 - val_loss: 4.2982e-04 - val_mse: 4.2982e-04 - 69ms/epoch - 11ms/step
Epoch 68/300
6/6 - 0s - loss: 4.1523e-04 - mse: 4.1523e-04 - val_loss: 4.2544e-04 - val_mse: 4.2544e-04 - 70ms/epoch - 12ms/step
Epoch 69/300
6/6 - 0s - loss: 4.1370e-04 - mse: 4.1370e-04 - val_loss: 4.2431e-04 - val_mse: 4.2431e-04 - 69ms/epoch - 12ms/step
Epoch 70/300
6/6 - 0s - loss: 4.1226e-04 - mse: 4.1226e-04 - val_loss: 4.2304e-04 - val_mse: 4.2304e-04 - 62ms/epoch - 10ms/step
Epoch 71/300
6/6 - 0s - loss: 4.1179e-04 - mse: 4.1179e-04 - val_loss: 4.2705e-04 - val_mse: 4.2705e-04 - 62ms/epoch - 10ms/step
Epoch 72/300
6/6 - 0s - loss: 4.1078e-04 - mse: 4.1078e-04 - val_loss: 4.2718e-04 - val_mse: 4.27

Epoch 129/300
6/6 - 0s - loss: 3.3989e-04 - mse: 3.3989e-04 - val_loss: 3.4866e-04 - val_mse: 3.4866e-04 - 74ms/epoch - 12ms/step
Epoch 130/300
6/6 - 0s - loss: 3.4015e-04 - mse: 3.4015e-04 - val_loss: 3.4672e-04 - val_mse: 3.4672e-04 - 70ms/epoch - 12ms/step
Epoch 131/300
6/6 - 0s - loss: 3.3911e-04 - mse: 3.3911e-04 - val_loss: 3.4740e-04 - val_mse: 3.4740e-04 - 73ms/epoch - 12ms/step
Epoch 132/300
6/6 - 0s - loss: 3.3815e-04 - mse: 3.3815e-04 - val_loss: 3.4896e-04 - val_mse: 3.4896e-04 - 69ms/epoch - 11ms/step
Epoch 133/300
6/6 - 0s - loss: 3.3802e-04 - mse: 3.3802e-04 - val_loss: 3.4485e-04 - val_mse: 3.4485e-04 - 64ms/epoch - 11ms/step
Epoch 134/300
6/6 - 0s - loss: 3.3503e-04 - mse: 3.3503e-04 - val_loss: 3.4328e-04 - val_mse: 3.4328e-04 - 63ms/epoch - 10ms/step
Epoch 135/300
6/6 - 0s - loss: 3.3264e-04 - mse: 3.3264e-04 - val_loss: 3.4185e-04 - val_mse: 3.4185e-04 - 62ms/epoch - 10ms/step
Epoch 136/300
6/6 - 0s - loss: 3.3246e-04 - mse: 3.3246e-04 - val_loss: 3.4039e-04 - val_m

6/6 - 0s - loss: 2.7973e-04 - mse: 2.7973e-04 - val_loss: 2.8725e-04 - val_mse: 2.8725e-04 - 72ms/epoch - 12ms/step
Epoch 193/300
6/6 - 0s - loss: 2.7881e-04 - mse: 2.7881e-04 - val_loss: 2.9123e-04 - val_mse: 2.9123e-04 - 77ms/epoch - 13ms/step
Epoch 194/300
6/6 - 0s - loss: 2.7959e-04 - mse: 2.7959e-04 - val_loss: 2.8523e-04 - val_mse: 2.8523e-04 - 102ms/epoch - 17ms/step
Epoch 195/300
6/6 - 0s - loss: 2.7802e-04 - mse: 2.7802e-04 - val_loss: 2.8576e-04 - val_mse: 2.8576e-04 - 100ms/epoch - 17ms/step
Epoch 196/300
6/6 - 0s - loss: 2.7631e-04 - mse: 2.7631e-04 - val_loss: 2.9025e-04 - val_mse: 2.9025e-04 - 112ms/epoch - 19ms/step
Epoch 197/300
6/6 - 0s - loss: 2.7711e-04 - mse: 2.7711e-04 - val_loss: 2.9206e-04 - val_mse: 2.9206e-04 - 85ms/epoch - 14ms/step
Epoch 198/300
6/6 - 0s - loss: 2.7685e-04 - mse: 2.7685e-04 - val_loss: 2.8311e-04 - val_mse: 2.8311e-04 - 74ms/epoch - 12ms/step
Epoch 199/300
6/6 - 0s - loss: 2.7788e-04 - mse: 2.7788e-04 - val_loss: 2.8453e-04 - val_mse: 2.8453e

6/6 - 0s - loss: 2.3527e-04 - mse: 2.3527e-04 - val_loss: 2.3929e-04 - val_mse: 2.3929e-04 - 66ms/epoch - 11ms/step
Epoch 256/300
6/6 - 0s - loss: 2.3344e-04 - mse: 2.3344e-04 - val_loss: 2.3974e-04 - val_mse: 2.3974e-04 - 66ms/epoch - 11ms/step
Epoch 257/300
6/6 - 0s - loss: 2.3330e-04 - mse: 2.3330e-04 - val_loss: 2.3644e-04 - val_mse: 2.3644e-04 - 61ms/epoch - 10ms/step
Epoch 258/300
6/6 - 0s - loss: 2.3241e-04 - mse: 2.3241e-04 - val_loss: 2.4200e-04 - val_mse: 2.4200e-04 - 67ms/epoch - 11ms/step
Epoch 259/300
6/6 - 0s - loss: 2.3070e-04 - mse: 2.3070e-04 - val_loss: 2.4088e-04 - val_mse: 2.4088e-04 - 68ms/epoch - 11ms/step
Epoch 260/300
6/6 - 0s - loss: 2.2973e-04 - mse: 2.2973e-04 - val_loss: 2.3665e-04 - val_mse: 2.3665e-04 - 67ms/epoch - 11ms/step
Epoch 261/300
6/6 - 0s - loss: 2.2932e-04 - mse: 2.2932e-04 - val_loss: 2.3497e-04 - val_mse: 2.3497e-04 - 69ms/epoch - 12ms/step
Epoch 262/300
6/6 - 0s - loss: 2.2735e-04 - mse: 2.2735e-04 - val_loss: 2.3348e-04 - val_mse: 2.3348e-04

In [19]:
#calculate the input and the output

print(scaler_X.mean_)
print(scaler_X.scale_)

print(scaler_y.mean_)
print(scaler_y.scale_)

[-9.24516369e+00  2.07031250e-01  6.07638889e+03 -3.24074074e-02]
[3.40929820e+01 7.49435366e-01 3.14538187e+05 2.25202336e+00]
[-9.19380193  0.20726607]
[35.34013469  0.74951056]


In [20]:
transf_model1.save('transfer_model1.h5')